In [ ]:
import os
import re
import zipfile

import numpy as np
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

url = "https://github.com/srihari-humbarwadi/datasets/releases/download/v0.1.0/data.zip"
filename = os.path.join(os.getcwd(), "data.zip")
keras.utils.get_file(filename, url)


with zipfile.ZipFile("data.zip", "r") as z_fp:
  z_fp.extractall("./")

def swap_xy(boxes):
  return tf.stack([boxes[:, 1], boxes[:, 0], boxes[:, 3], boxes[:, 2]], axis=-1)

def convert_to_xywh(boxes):
  return tf.concat(
    [(boxes[..., :2] + boxes[..., 2:]) / 2.0, boxes[..., 2:] - boxes[..., :2]],
    axis=-1,
  )

def convert_to_corners(boxes):
  return tf.concat(
    [boxes[..., :2] - boxes[..., 2:] / 2.0, boxes[..., :2] + boxes[..., 2:] / 2.0],
    axis=-1,
  )

def compute_iou(boxes1, boxes2):
  boxes1_corners = convert_to_corners(boxes1)
  boxes2_corners = convert_to_corners(boxes2)
  lu = tf.maximum(boxes1_corners[:, None, :2], boxes2_corners[:, :2])
  rd = tf.maximum(boxes1_corners[:, None, :2], boxes2_corners[:, :2])
  intersection = tf.maximum(0.0, rd-lu)
  intersection_area = intersection[:, :, 0] * intersection[:, :, 1]
  boxes1_area = boxes1[:, 2] * boxes1[:, 3]
  boxes2_area = boxes2[:, 2] * boxes2[:, 3]
  union_area = tf.maximum(boxes1_area[:, None] + boxes2_area -intersection_area, 1e-8)

  return tf.clip_by_value(intersection_area / union_area, 0.0, 1.0)

def visualize_detections(image, boxes, classes, scores, figsize=(7, 7), linewidth=1
                         color=[0, 0, 1]):
  image = np.array(image, dtype=np.uint8)
  plt.figure(figsize=figsize)
  plt.axis("off")
  plt.imshow(image)
  ax = plt.gca()
  